# 🚀 AGI Core Hunter - MDL vs OOD (Optimized)

**Enhanced with GPU optimizations, robust error handling, and testing options.**

## 🎯 Experiment Options:
- **🏃 GPU Test**: 2 episodes (~2 min) - Validate setup
- **⚡ Mini**: Quick experiment (~5 min) - Fast validation  
- **🧪 Full**: Complete study (~30-45 min)

**⚠️ Runtime → Change runtime type → GPU (Tesla T4)**

In [ ]:
# 🔧 Enhanced Setup with GPU Optimization
print('🔍 GPU Check:')
!nvidia-smi --query-gpu=name,memory.total,memory.used --format=csv

print('\n🧹 Clean installation...')
!rm -rf agi-core-hunter 2>/dev/null || true
!git clone https://github.com/inkbytefo/agi-core-hunter.git
%cd agi-core-hunter

print('🚀 Installing optimized dependencies...')
!pip install --upgrade pip -q
!pip install -U "jax[cuda12]" -q
!pip install flax optax chex wandb tqdm matplotlib seaborn pandas numpy -q

print('🧪 Verifying setup...')
!python test_setup.py
print('✅ Setup complete!')

In [ ]:
# 🧪 GPU Validation
%cd experiments/01_mdl_vs_ood

print('🔥 GPU performance test...')
!python gpu_test.py

print('\n🏃 Quick training test (2 episodes):')
!python train.py --gpu-test
print('✅ GPU validation complete!')

In [ ]:
# 📊 WandB Setup with Fallback
import wandb, os

try:
    wandb.login()
    print('✅ WandB online mode')
except:
    print('🔄 Using offline mode')
    os.environ['WANDB_MODE'] = 'offline'

In [ ]:
# 🎯 Experiment Selection
import json

with open('manifest.json', 'r') as f:
    config = json.load(f)

print('🧪 EXPERIMENT OVERVIEW:')
print(f'Hypothesis: {config["experiment"]["hypothesis"]}')
print(f'Episodes: {config["training"]["total_episodes"]}')
print('Agents:')
for agent in config['agents']:
    print(f'  {agent["name"]}: β={agent["config"]["beta"]}')

# Choose experiment type (modify this)
EXPERIMENT_CHOICE = 3  # 1=GPU Test, 2=Mini, 3=Full

if EXPERIMENT_CHOICE == 1:
    cmd = 'python train.py --gpu-test'
    desc = 'GPU Test (~2 min)'
elif EXPERIMENT_CHOICE == 2:
    cmd = 'python train.py --manifest mini-test.json'
    desc = 'Mini Experiment (~5 min)'
else:
    cmd = 'python train.py --manifest manifest.json'
    desc = 'Full Experiment (~30-45 min)'

print(f'\n🎯 Selected: {desc}')
print(f'Command: {cmd}')

In [ ]:
# 🚀 Execute Experiment
import time, subprocess

start_time = time.time()
print(f'🚀 Starting {desc}...')
print('='*50)

# Real-time execution
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, 
                          stderr=subprocess.STDOUT, universal_newlines=True)

for line in process.stdout:
    print(line.rstrip())

process.wait()
duration = time.time() - start_time

print('='*50)
print(f'🎉 Completed in {duration/60:.1f} minutes')
success = process.returncode == 0
print('✅ Success!' if success else '⚠️ Finished with warnings')

In [ ]:
# 📊 Results Analysis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os

# Find results file
result_files = glob.glob('results*.json')
if result_files:
    latest_file = max(result_files, key=os.path.getctime)
    print(f'📂 Using: {latest_file}')
    
    with open(latest_file, 'r') as f:
        results = json.load(f)
    
    # Process data
    data = []
    for agent, agent_data in results.items():
        beta = agent_data['agent_config']['beta']
        std_success = agent_data['standard_success_rate']
        ood_ratios = [v for k, v in agent_data['ood_performance_ratios'].items() 
                     if k.endswith('_performance_ratio')]
        avg_ood = np.mean(ood_ratios) if ood_ratios else 0
        
        data.append({
            'Agent': agent, 'Beta': beta,
            'Standard': std_success, 'OOD_Ratio': avg_ood
        })
    
    df = pd.DataFrame(data).sort_values('Beta')
    print('\n📋 RESULTS:')
    print(df.round(3))
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Beta vs OOD
    ax1.scatter(df['Beta'], df['OOD_Ratio'], s=100, c='red')
    for i, agent in enumerate(df['Agent']):
        ax1.annotate(agent.replace('MDL_', ''), 
                    (df['Beta'].iloc[i], df['OOD_Ratio'].iloc[i]))
    ax1.set_xlabel('Beta')
    ax1.set_ylabel('OOD Performance Ratio')
    ax1.set_title('🎯 Main Hypothesis Test')
    ax1.grid(True, alpha=0.3)
    
    # Performance comparison
    x_pos = range(len(df))
    ax2.bar([x-0.2 for x in x_pos], df['Standard'], 0.4, label='Standard')
    ax2.bar([x+0.2 for x in x_pos], df['OOD_Ratio'], 0.4, label='OOD')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels([a.replace('MDL_', '') for a in df['Agent']])
    ax2.legend()
    ax2.set_title('📊 Performance Comparison')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Hypothesis test
    if len(df) >= 2:
        correlation = np.corrcoef(df['Beta'], df['OOD_Ratio'])[0, 1]
        print(f'\n📈 Correlation: {correlation:.3f}')
        
        if correlation > 0.3:
            print('✅ HYPOTHESIS SUPPORTED!')
        elif correlation < -0.3:
            print('❌ HYPOTHESIS REJECTED!')
        else:
            print('❓ HYPOTHESIS INCONCLUSIVE')
else:
    print('❌ No results found!')

In [ ]:
# 💾 Download Results
from google.colab import files

if 'latest_file' in locals():
    try:
        # Create package
        package = f'agi_results_{time.strftime("%Y%m%d_%H%M%S")}'
        !mkdir -p {package}
        !cp {latest_file} {package}/results.json
        !cp manifest.json {package}/config.json 2>/dev/null || true
        
        # Summary
        if 'correlation' in locals():
            summary = {
                'experiment': desc,
                'duration_min': round(duration/60, 2),
                'correlation': float(correlation),
                'agents': len(df),
                'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
            }
            
            with open(f'{package}/summary.json', 'w') as f:
                json.dump(summary, f, indent=2)
        
        # Download
        !zip -r {package}.zip {package}/
        files.download(f'{package}.zip')
        print(f'✅ Downloaded: {package}.zip')
        
    except Exception as e:
        print(f'⚠️ Package error: {e}')
        files.download(latest_file)
        print(f'✅ Downloaded: {latest_file}')
else:
    print('❌ No results to download')